In [0]:
# Databricks notebook: 02_exploratory_analysis.py
# Purpose: Notebook for exploration and charts. Assumes delta table madsc102.us_accidents_clean exists.


DATABASE = 'madsc102'
TABLE = 'usaccidents_volume'
FULL_TABLE = f"{DATABASE}.{TABLE}"


# 1) Basic counts and distributions
print('Total rows:', spark.table(FULL_TABLE).count())
display(spark.sql(f"SELECT severity, COUNT(*) as cnt FROM {FULL_TABLE} GROUP BY severity ORDER BY severity"))


# 2) Daily trend (top 100 days)
daily = spark.sql(f"SELECT start_date, COUNT(*) as total_accidents, ROUND(AVG(duration_minutes),2) as avg_duration FROM {FULL_TABLE} GROUP BY start_date ORDER BY start_date DESC LIMIT 100")
display(daily)


# 3) Hourly pattern
hourly = spark.sql(f"SELECT start_hour, COUNT(*) as cnt FROM {FULL_TABLE} GROUP BY start_hour ORDER BY start_hour")
display(hourly)


# 4) Top states by accidents and severity
states = spark.sql(f"SELECT state, COUNT(*) as accidents, ROUND(AVG(severity),2) as avg_severity FROM {FULL_TABLE} GROUP BY state ORDER BY accidents DESC LIMIT 20")
display(states)


# 5) Geo aggregates for heatmap
geo = spark.sql(f"SELECT ROUND(latitude,3) as lat_r, ROUND(longitude,3) as lon_r, COUNT(*) as cnt FROM {FULL_TABLE} GROUP BY lat_r, lon_r HAVING cnt > 5 ORDER BY cnt DESC LIMIT 1000")
display(geo)


# 6) Save sample to CSV for external viz if needed
# 1. Define the correct, supported Volume Path for the CSV
# Replace 'workspace' and 'default' with your actual catalog and schema if necessary
VOLUME_PATH = "/Volumes/workspace/default/usaccidents_volume/us_accidents_sample.csv"

# 2. Sample the data using Spark and write it directly as CSV
# NOTE: If you need a single CSV file, you must first repartition to 1.
print(f"Sampling and writing CSV to: {VOLUME_PATH}")

(
    spark.table(FULL_TABLE)
    .sample(0.01, seed=42)
    .limit(10000)
    .repartition(1)  # Forces Spark to output the sample into a single CSV file
    .write
    .mode("overwrite")
    .option("header", "true")
    .csv(VOLUME_PATH)
)

print("Sample data successfully written.")

Total rows: 7061773


severity,cnt
1,65178
2,5680330
3,1137268
4,178997


start_date,total_accidents,avg_duration
2023-03-31,1983,2.800463147E7
2023-03-30,1697,2.800285294E7
2023-03-29,2594,2.800155193E7
2023-03-28,2404,2.800015318E7
2023-03-27,2084,2.799868243E7
2023-03-26,1623,2.799729098E7
2023-03-25,2094,2.799583221E7
2023-03-24,2795,2.799443113E7
2023-03-23,2452,2.799296833E7
2023-03-22,2565,2.799146628E7


start_hour,cnt
0,101263
1,87536
2,83611
3,74963
4,141136
5,199384
6,353926
7,513604
8,511245
9,325305


state,accidents,avg_severity
CA,1559770,2.15
FL,830829,2.13
TX,537044,2.22
SC,348798,2.1
NY,326476,2.25
NC,309539,2.13
PA,269875,2.2
VA,268379,2.27
MN,181426,2.16
OR,165078,2.11


lat_r,lon_r,cnt
34.859,-82.26,1551
37.808,-122.367,1474
33.941,-118.097,1439
25.964,-80.166,1018
33.745,-84.39,947
33.925,-118.21,876
33.913,-118.18,864
33.722,-84.395,825
34.01,-117.823,769
26.011,-80.167,765


Sampling and writing CSV to: /Volumes/workspace/default/usaccidents_volume/us_accidents_sample.csv
Sample data successfully written.


In [0]:
%sql
-- daily_accidents.sql
SELECT start_date,
       COUNT(*) AS total_accidents,
       ROUND(AVG(duration_minutes),2) AS avg_duration_min
FROM madsc102.usaccidents_volume
GROUP BY start_date
ORDER BY start_date DESC
LIMIT 100;

start_date,total_accidents,avg_duration_min
2023-03-31,1983,2.800463147E7
2023-03-30,1697,2.800285294E7
2023-03-29,2594,2.800155193E7
2023-03-28,2404,2.800015318E7
2023-03-27,2084,2.799868243E7
2023-03-26,1623,2.799729098E7
2023-03-25,2094,2.799583221E7
2023-03-24,2795,2.799443113E7
2023-03-23,2452,2.799296833E7
2023-03-22,2565,2.799146628E7


In [0]:
%sql
-- top_states_severity.sql
SELECT state,
       COUNT(*) AS accidents,
       ROUND(AVG(severity),2) AS avg_severity,
       ROUND(PERCENTILE(duration_minutes, 0.5),2) AS median_duration_min
FROM madsc102.usaccidents_volume
GROUP BY state
HAVING COUNT(*) > 1000
ORDER BY avg_severity DESC
LIMIT 10;

state,accidents,avg_severity,median_duration_min
GA,151585,2.49,2.648693728E7
WI,32712,2.46,2.633056197E7
RI,15383,2.46,2.601875947E7
KY,29901,2.45,2.614925473E7
CO,83534,2.43,2.650562008E7
IA,24278,2.41,2.681244248E7
MO,70600,2.4,2.67965031E7
IN,62012,2.39,2.670875658E7
IL,161831,2.38,2.62862055E7
NM,9515,2.37,2.64892222E7


In [0]:
{
  "name": "madsc102-us-accidents-ingestion",
  "tasks": [
    {
      "task_key": "ingest_clean",
      "notebook_task": {
        "notebook_path": "/Repos/<your-repo>/notebooks/01_ingest_clean_write_delta"
      }
    }
  ],
  "schedule": {
    "quartz_cron_expression": "0 0 3 ? * * *",
    "timezone_id": "UTC"
  }
}

{'name': 'madsc102-us-accidents-ingestion',
 'tasks': [{'task_key': 'ingest_clean',
   'notebook_task': {'notebook_path': '/Repos/<your-repo>/notebooks/01_ingest_clean_write_delta'}}],
 'schedule': {'quartz_cron_expression': '0 0 3 ? * * *', 'timezone_id': 'UTC'}}

In [0]:
%sql
